In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch3/users/d/djl34"

In [2]:
client = Client()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39440 instead
  warnings.warn(


In [3]:
file_directory = "single_feature/nonsense/"

all_chrom_set = [str(x) for x in range(1, 23)]
chrom_set = all_chrom_set

header_list = ["nonsense_below_5_pivot_", "nonsense_above_5_pivot_"]

file_list = [os.path.join(KL_data_dir, "raklette_output/" + file_directory + header + "chr_" + chrom + "_chunksize_1000000_covonly_lr_0.01_gamma_0.5_epoch_800_covprior_0.1_KL.tsv") for chrom in chrom_set for header in header_list]

In [48]:
len(file_list)

44

In [26]:
df = dd.read_csv(file_list, sep = "\t")

df = df.compute()

In [27]:
shet = pd.read_csv(pd_data_dir + "/shet_gnomAD_Roulette_0.01cutoff_v2.csv")

ad = pd.read_csv(KL_data_dir + "/clinical/CGD_AD_HQ_ENSG.tsv", sep = "\t")

ad["AD"] = 1

shet = shet.rename({"Gene_stable_ID": "Gene"}, axis = 1)

df = df.merge(shet[["Gene", "mean_s_het"]], on = "Gene", how = "inner")

df = df.merge(ad[["Gene", "AD"]], on = "Gene", how = "left")

df = df.fillna(0)

In [35]:
df['KL_fw_decile'] = pd.qcut(df['KL_fw'], 10, labels=False)
df['KL_rv_decile'] = pd.qcut(df['KL_rv'], 10, labels=False)
df['mean_s_het_decile'] = pd.qcut(df['mean_s_het'], 10, labels=False)

In [47]:
for column in ['KL_fw_decile', 'KL_rv_decile', 'mean_s_het_decile']:
    print(column)
    df_decile = df[df[column] == 9]
    
    print(len(df_decile[df_decile["AD"] == 1])/df["AD"].sum())

KL_fw_decile
0.36607142857142855
KL_rv_decile
0.38839285714285715
mean_s_het_decile
0.45089285714285715


In [37]:
df["KL_fw_decile"] == 0

array([2, 7, 4, 3, 1, 5, 9, 6, 0, 8])

In [45]:
df.sort_values("KL_fw")

,Gene,KL_fw,KL_rv,mean_s_het,AD,KL_fw_decile,KL_rv_decile,mean_s_het_decile
12725,ENSG00000178852,0.009174,0.009350,0.005927,0.0,0,0,0
7553,ENSG00000187922,0.009536,0.009832,0.010814,0.0,0,0,1
14298,ENSG00000105136,0.009744,0.009952,0.022303,0.0,0,0,4
15628,ENSG00000183569,0.010672,0.010955,0.004043,0.0,0,0,0
10826,ENSG00000170054,0.010904,0.011192,0.004221,0.0,0,0,0
...,...,...,...,...,...,...,...,...
7041,ENSG00000104517,5.321619,3.832190,0.612819,0.0,9,9,9
7898,ENSG00000107581,5.332687,3.832646,0.311606,0.0,9,9,9
4611,ENSG00000164190,5.770188,3.846273,1.000000,1.0,9,9,9
5052,ENSG00000242419,5.911559,3.078976,0.136610,0.0,9,9,8
